<a href="https://colab.research.google.com/github/tilakparth/Automatic-Image-Captioning/blob/main/Automatic_Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt